In [9]:
import os
import torch
import torch.cuda
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import datasets,models,transforms

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

class GroupNorm32(torch.nn.GroupNorm):
    def __init__(self, num_channels, num_groups=2, **kargs):
        super().__init__(num_groups, num_channels, **kargs)

if __name__ == '__main__':
    """# Dataset transformation and loading"""
    
    #Transformations are RandomCrop, RandomHorizontalFlip and Normalization
    transform = transforms.Compose([transforms.RandomHorizontalFlip(0.5),
                                    transforms.RandomCrop(32),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
    #Dataset (train and test) downloads and transforms appliance
    image_train_set = datasets.CIFAR10(root="./data", train = True, download=True, transform = transform)
    image_test_set = datasets.CIFAR10(root="./data", train = False, download=True, transform = transform)
    
    n_train = len(image_train_set)
    indices = list(range(n_train))
    np.random.shuffle(indices) #Randomize indices to avoid overfitting
    split_mask = int(np.floor(0.2*n_train)) #Create a the threshold to split train in train and validation
    train_indices,valid_indices = indices[split_mask:],indices[:split_mask]
    
    #Initiliazation of samplers to divide train and valid into batches
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(valid_indices)
    
    #Loading sets into dataloaders
    train_loader = DataLoader(image_train_set, batch_size = 20, sampler = train_sampler, num_workers = 3)
    valid_loader = DataLoader(image_train_set, batch_size = 20, sampler = valid_sampler, num_workers = 3)
    test_loader = DataLoader(image_test_set, batch_size = 20, num_workers = 3)
    
    #Declare the list of classes
    labels_list = ["Plane","Car","Bird","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]
    
    optim_parameters = {"lr":[1e-2],"momentum":[0,0.5,0.9],"weight_decay":[0,0.01,0.1]}
    norm_layers = [None,GroupNorm32]
    tuning_results = {"lr":[],"mom":[],"wd":[],"norm":[],"train_loss":[],"test_loss":[],"global_accuracy":[]}

    for norms in norm_layers:
      for lr_param in optim_parameters["lr"]:
        for mom_param in optim_parameters["momentum"]:
          for wd_param in optim_parameters["weight_decay"]:
              """# Model Initialization"""
              #Definition of ResNet50
              if norms == None:
                model = models.resnet50(pretrained = True, progress = True, norm_layer = norms)
              else:
                model = models.resnet50(pretrained = False, progress = True, norm_layer = norms)

              #model = ResNet50(num_classes = len(labels_list), pretrained = True)
              model.cuda() #Passes the model calculation to the GPU
              
              #Loss Criterion = CrossEntropy
              loss_criterion = nn.CrossEntropyLoss()
              #Optimizer = SGD
              optim = torch.optim.SGD(model.parameters(), lr = lr_param, momentum = mom_param, weight_decay = wd_param)
              
              """# Training and Validation"""
              
              #Declaring number of epochs
              epochs = [*range(3)]
              #Declaring list for the losses
              train_loss_list = []
              valid_loss_min = np.Inf #To keep track of the validation loss
              
              for epoch in tqdm(range(1,(len(epochs)+1))):
              
                #To keep track of training and validation loss
                train_loss = 0.0
                valid_loss = 0.0
              
                ##MODEL TRAINING##
                model.train()
                for data,target in train_loader:
                  #Use GPU for tensors
                  data, target = data.cuda(), target.cuda()
                  #Clear optimizer gradients
                  optim.zero_grad()
                  #Computation of predicted output
                  output = model(data)
                  #Calculating batch loss
                  loss = loss_criterion(output,target)
                  #Backward
                  loss.backward()
                  #Optimization step
                  optim.step()
                  #Training loss update
                  train_loss += loss.item()*data.size(0)
              
                
                ##MODEL VALIDATION##
                model.eval()
                for data,target in valid_loader:
                  #Use GPU for tensors
                  data,target = data.cuda(), target.cuda()
                  #Computation of predicted output
                  output = model(data)
                  #Loss computation
                  loss = loss_criterion(output,target)
                  valid_loss += loss.item()*data.size(0)
              
                #Calculation of Average Losses
                train_loss = train_loss/len(train_loader.dataset)
                valid_loss = valid_loss/len(valid_loader.dataset)
                train_loss_list.append(train_loss)
              
                #Print training/validation statistics 
                print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
              
                #Save model for decreasing losses
                if valid_loss <= valid_loss_min:
                  print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
                  torch.save(model.state_dict(), 'model_cifar.pt')
                  valid_loss_min = valid_loss
              
                #Printing Epochs Results
                #plt.plot(epochs, train_loss_list)
                #plt.xlabel("Epoch")
                #plt.ylabel("Loss")
                #plt.title("Performance of Model 1")
                #plt.show()
                
                """# Testing"""
                
              #Tracking of test loss
              test_loss = 0.0
              class_correct = list(0. for i in range(len(labels_list)))
              class_total = list(0. for i in range(len(labels_list)))
              train_on_gpu = torch.cuda.is_available()
                
              #Testing the model
              model.eval()
              for data,target in test_loader:
                #Move to GPU if is available
                data,target = data.cuda(),target.cuda()
                #Output computation
                output = model(data)
                #Test loss computation
                loss = loss_criterion(output,target)
                #Update loss
                test_loss += loss.item()*data.size(0)
                #Convert output to probabilities
                _, pred = torch.max(output,1)
                #Compare prediction to target
                correct_tensor = pred.eq(target.data.view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
                #Compute test accuracy
                for i in range(2*len(labels_list)):
                  label = target.data[i]
                  class_correct[label] += correct[i].item()
                  class_total[label] += 1
                
              test_loss = test_loss/len(test_loader.dataset)
              print('Test Loss: {:.6f}\n'.format(test_loss))
              
              """# Visualize Accuracy Results"""
              
              print(f"Configuration used: lr = {lr_param}, momentum = {mom_param}, weight_decay = {wd_param}, norm = {norms} (If none -> Batch)")


              for i in range(len(labels_list)):
                  if class_total[i] > 0:
                      print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                          labels_list[i], 100 * class_correct[i] / class_total[i],
                          np.sum(class_correct[i]), np.sum(class_total[i])))
                  else:
                      print('Test Accuracy of %5s: N/A (no training examples)' % (labels_list[i]))
              
              print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
                  100. * np.sum(class_correct) / np.sum(class_total),
                  np.sum(class_correct), np.sum(class_total)))
              
              tuning_results["lr"].append(lr_param)
              tuning_results["mom"].append(mom_param)
              tuning_results["wd"].append(wd_param)
              tuning_results["train_loss"].append(train_loss)
              tuning_results["test_loss"].append(test_loss)
              tuning_results["global_accuracy"].append(100. * np.sum(class_correct) / np.sum(class_total))

    print(tuning_results)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 	Training Loss: 1.080258 	Validation Loss: 0.144626
Validation loss decreased (inf --> 0.144626).  Saving model ...


 33%|███▎      | 1/3 [01:11<02:22, 71.19s/it]

Epoch: 2 	Training Loss: 0.556761 	Validation Loss: 0.116790
Validation loss decreased (0.144626 --> 0.116790).  Saving model ...


...

 67%|██████▋   | 2/3 [03:22<01:41, 101.33s/it]

Epoch: 2 	Training Loss: 2.040762 	Validation Loss: 0.505571
Validation loss decreased (0.532371 --> 0.505571).  Saving model ...


100%|██████████| 3/3 [05:04<00:00, 101.57s/it]

Epoch: 3 	Training Loss: 2.042596 	Validation Loss: 0.539828


Test Loss: 2.694690

Configuration used: lr = 0.01, momentum = 0.9, weight_decay = 0.1, norm = <class '__main__.GroupNorm32'> (If none -> Batch)
Test Accuracy of Plane:  0% ( 0/1000)
Test Accuracy of   Car:  0% ( 0/1000)
Test Accuracy of  Bird:  0% ( 0/1000)
Test Accuracy of   Cat:  0% ( 0/1000)
Test Accuracy of  Deer:  0% ( 0/1000)
Test Accuracy of   Dog:  0% ( 0/1000)
Test Accuracy of  Frog:  0% ( 0/1000)
Test Accuracy of Horse:  0% ( 0/1000)
Test Accuracy of  Ship: 100% (1000/1000)
Test Accuracy of Truck:  0% ( 0/1000)

Test Accuracy (Overall): 10% (1000/10000)
{'lr': [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], 'mom': [0, 0, 0, 0.5, 0.5, 0.5, 0.9, 0.9, 0.9, 0, 0, 0, 0.5, 0.5, 0.5, 0.9, 0.9, 0.9], 'wd': [0, 0.01, 0.1, 0, 0.01, 0.1, 0, 0.01, 0.1, 0, 0.01, 0.1, 0, 0.01, 0.1, 0, 0.01, 0.1], 'norm': [], 'train_loss': [0.4482509360939264, 0.46819082063436507, 1.3171357173442841, 0.6065801052510739, 0.9873050760984421, 1.545